# 装载module

所有函数,类型以及类型类都是 Prelude 模块的一部分，它缺省自动装载

运行 import Data.List，这样一来 Data.List 中包含的所有函数就都进入了全局命名空间。也就是说，你可以在代码的任意位置调用这些函数.

Data.List 模块中有个 nub 函数，它可以筛掉一个 List 中的所有重复元素

In [1]:
import Data.List

numUnique :: (Eq a) => [a] -> Int
numUnique = length . nub -- 等价于(\xs -> length (nub xs))

In [2]:
import Data.List

numUnique :: (Eq a) => [a] -> Int
numUnique = (\xs -> length (nub xs)) 

In [3]:
numUnique [1,2,3]

3

## gchi 中装载模块：直接:m+

ghci> :m + Data.List Data.Map Data.Set

- 指定装载函数：直接后面加括号

In [4]:
import Data.List (nub, sort)

- hiding: 只包含除去某函数之外的其它函数，这在避免多个模块中函数的命名冲突很有用

In [5]:
import Data.List hiding (nub)

- qualified import : 避免命名冲突

In [10]:
import Data.Map (filter)

In [11]:
filter (>5) [1, 2, 3] --与Prelude命名冲突，应该用qualified import

<interactive>:1:1: error:
    Ambiguous occurrence ‘filter’
    It could refer to either ‘Data.Map.filter’, imported from ‘Data.Map’ at <interactive>:1:18-23 (and originally defined in ‘Data.Map.Internal’)
                          or ‘IHaskellPrelude.filter’, imported from ‘Prelude’ (and originally defined in ‘GHC.List’)

这时候就可以用qualified import

In [8]:
import qualified Data.Map (filter)

In [12]:
filter (>5) [1,2,3]

<interactive>:1:1: error:
    Ambiguous occurrence ‘filter’
    It could refer to either ‘Data.Map.filter’, imported from ‘Data.Map’ at <interactive>:1:18-23 (and originally defined in ‘Data.Map.Internal’)
                          or ‘IHaskellPrelude.filter’, imported from ‘Prelude’ (and originally defined in ‘GHC.List’)

In [22]:
a = Data.Map.fromList (map makePair [1..3]) where makePair x = (x,[x])
a

fromList [(1,[1]),(2,[2]),(3,[3])]

In [23]:
Data.Map.filter (<[2]) a

fromList [(1,[1])]

- 别名

In [24]:
import qualified Data.Map as M

In [25]:
M.filter (<[2]) a

fromList [(1,[1])]

## Data.List

Prelude 模块出于方便起见，导出了几个 Data.List 里的函数(map, filter...)。因为这几个函数是直接引用自 Data.List，所以就无需使用 qualified import

- intersperse

取一个元素与 List 作参数，并将该元素置于 List 中每对元素的中间。如下是个例子

In [30]:
intersperse "." "MONKEY"

<interactive>:1:17: error:
    • Couldn't match type ‘Char’ with ‘String’
      Expected type: [String]
        Actual type: String
    • In the second argument of ‘intersperse’, namely ‘"MONKEY"’
      In the expression: intersperse "." "MONKEY"
      In an equation for ‘it’: it = intersperse "." "MONKEY"

In [31]:
intersperse '.' "MONKEY" -- "" 是list（String）

"M.O.N.K.E.Y"

In [29]:
intersperse 1 [1,2,3]

[1,1,2,1,3]

- intercalate 

取两个 List 作参数。它会将第一个 List 交叉插入第二个 List 中间，并返回一个 List

In [33]:
intercalate " " ["hey", "there", "guys"]

Line 1: Use unwords
Found:
intercalate " "
Why not:
unwords

"hey there guys"

In [34]:
intercalate [0,0,0] [[1,2,3],[4,5,6],[7,8,9]] 

[1,2,3,0,0,0,4,5,6,0,0,0,7,8,9]

- transpose 

transpose 函数可以反转一组 List 的 List。你若把一组 List 的 List 看作是个 2D 的矩阵，那 transpose 的操作就是将其列为行

In [35]:
transpose [[1,2,3], [4,5,6],[7,8,9]]

[[1,4,7],[2,5,8],[3,6,9]]

In [36]:
transpose ["hey","there","guys"]

["htg","ehu","yey","rs","e"]

利用transpose 相加这三个多项式：3x<sup>2</sup> + 5x + 9，10x<sup>3</sup> + 9 和 8x<sup>3</sup> + 5x<sup>2</sup> + x - 1 

In [39]:
transpose[ [0, 3, 5, 9], [10, 0, 0, 9], [8, 5, 1, -1]]
map sum $ transpose[ [0, 3, 5, 9], [10, 0, 0, 9], [8, 5, 1, -1]]

[[0,10,8],[3,0,5],[5,0,1],[9,9,-1]]

[18,8,6,17]

- foldl 与foldl1 的严格版

foldl' 和 foldl1' 是它们各自惰性实现的严格版本。在用 fold 处理较大的 List 时，经常会遇到堆栈溢出的问题。而这罪魁祸首就是 fold 的惰性: 在运行 fold 时，累加器的值并不会被立即更新，而是做一个"在必要时会取得所需的结果"的承诺。每过一遍累加器，这一行为就重复一次。而所有的这堆"承诺"最终就会塞满你的堆栈。

严格的 fold 就不会有这一问题，它们不会作"承诺"，而是直接计算中间值的结果并继续运行下去。如果用惰性 fold 时经常遇到溢出错误，就应换用它们的严格版

- concat  把一组 List 连接为一个 List

In [46]:
concat ["abc", "123"]
concat [[3,4,5],[2,3,4],[2,1,1]] 


"abc123"

[3,4,5,2,3,4,2,1,1]

它相当于移除一级嵌套。若要彻底地连接两层嵌套其中的元素，你得 concat 它两次才行

In [48]:
concat [[[2,3],[3,4,5],[2]],[[2,3],[3,4]]]

[[2,3],[3,4,5],[2],[2,3],[3,4]]

In [51]:
concat $ concat [[[2,3],[3,4,5],[2]],[[2,3],[3,4]]] 

Line 1: Use ++
Found:
concat [[[2, 3], [3, 4, 5], [2]], [[2, 3], [3, 4]]]
Why not:
([[2, 3], [3, 4, 5], [2]] ++ [[2, 3], [3, 4]])

[2,3,3,4,5,2,2,3,3,4]

- concatMap

concatMap 函数与 map 一个 List 之后再 concat 它

In [52]:
concatMap (replicate 4) [1..3]  

[1,1,1,1,2,2,2,2,3,3,3,3]

- and，or

and: 取一组布尔值 List 作参数。只有其中的值全为 True 的情况下才会返回 True

or: 取一组布尔值 List 作参数。其中的值有 True 的情况下就返回 True

In [56]:
and $ map (>4) [5,6,7,8] 
and $ map (==4) [4,4,4,3,4]
or $ map (>4) [5,6,7,8] 
or $ map (==4) [4,4,4,3,4]

Line 1: Use all
Found:
and $ map (> 4) [5, 6, 7, 8]
Why not:
all (> 4) [5, 6, 7, 8]Line 1: Use all
Found:
and $ map (== 4) [4, 4, 4, 3, 4]
Why not:
all (== 4) [4, 4, 4, 3, 4]Line 1: Use any
Found:
or $ map (> 4) [5, 6, 7, 8]
Why not:
any (> 4) [5, 6, 7, 8]Line 1: Use any
Found:
or $ map (== 4) [4, 4, 4, 3, 4]
Why not:
any (== 4) [4, 4, 4, 3, 4]

True

False

True

True

可以看出，用all也没问题

- any and all

any 和 all 取一个限制条件和一组布尔值 List 作参数，检查是否该 List 的某个元素或每个元素都符合该条件。通常较 map 一个 List 到 and 或 or 而言，使用 any 或 all 会更多些

- iterate

iterate 取一个函数和一个值作参数。它会用该值去调用该函数并用所得的结果再次调用该函数，产生一个无限的 List.

ghci> take 10 \$ iterate (*2) 1  
[1,2,4,8,16,32,64,128,256,512]  
ghci> take 3 \$ iterate (++ "haha") "haha"  
["haha","hahahaha","hahahahahaha"]  

- splitAt

splitAt 取一个 List 和数值作参数，将该 List 在特定的位置断开。返回一个包含两个 List 的二元组.

In [62]:
splitAt 3 "heyman"
splitAt 100 "heyman"
splitAt (-3) "heyman"
let (a,b) = splitAt 3 "foobar" in b ++ a 

("hey","man")

("heyman","")

("","heyman")

"barfoo"

## takeWhile 

这一函数十分的实用。它从一个 List 中取元素，一旦遇到不符合条件的某元素就停止

In [63]:
takeWhile (>3) [6,5,4,3,2,1,2,3,4,5,4,3,2,1]

[6,5,4]

In [67]:
takeWhile (/=' ') "This is me"

"This"

求所有三次方小于 1000 的数的和

In [71]:
sum $ takeWhile (<10000) $ map (^3) [1..100]

53361

## dropWhile 

dropWhile 与 takeWhile 此相似，不过它是扔掉符合条件的元素。一旦限制条件返回 False，它就返回 List 的余下部分。方便实用!

In [72]:
dropWhile (>3) [6,5,4,3,2,1,2,3,4,5,4,3,2,1]

[3,2,1,2,3,4,5,4,3,2,1]

In [73]:
dropWhile (/=' ') "This is me"

" is me"

给一 Tuple 组成的 List，这 Tuple 的首项表示股票价格，第二三四项分别表示年,月,日。我们想知道它是在哪天首次突破 $1000 的!

In [76]:
let stock = [(994.4,2008,9,1),(995.2,2008,9,2),(999.2,2008,9,3),(1001.4,2008,9,4),(998.3,2008,9,5)]
head $ dropWhile (\ (val, yy, mm, dd) -> val < 1000) stock

(1001.4,2008,9,4)

- span

span 与 takeWhile 有点像，只是它返回两个 List。第一个 List 与同参数调用 takeWhile 所得的结果相同，第二个 List 就是原 List 中余下的部分

In [79]:
let (fw, rest) = span (/=' ') "This is a sentence" in "First word:" ++ fw ++ ", the rest:" ++ rest

"First word:This, the rest: is a sentence"

- break

span 是在条件首次为 False 时断开 List，而 break 则是在条件首次为 True 时断开 List。break p 与 span (not . p) 是等价的.

In [82]:
break (==4) [1,2,3,4,5,6,7]
span (/=4) [1..7]

([1,2,3],[4,5,6,7])

([1,2,3],[4,5,6,7])

- sort

sort 可以排序一个 List，因为只有能够作比较的元素才可以被排序，所以这一 List 的元素必须是 Ord 类型类的实例类型

In [86]:
sort [1, 23, 4, 55]
sort "helloworld"

[1,4,23,55]

"dehllloorw"

- group

group 取一个 List 作参数，并将其中相邻并相等的元素各自归类，组成一个个子 List

In [87]:
group [1, 1, 1, 2, 2, 3 ,3 ,3, 4, 5 ,6]

[[1,1,1],[2,2],[3,3,3],[4],[5],[6]]

若在 group 一个 List 之前给它排序就可以得到每个元素在该 List 中的出现次数

In [91]:
map (\ l@(x:xs) -> (x, length l))  . group $ sort [1,1,1,2,2,2,3,3,4]

[(1,3),(2,3),(3,2),(4,1)]

- inits and tails

inits 和 tails 与 init 和 tail 相似，只是它们会递归地调用自身直到什么都不剩

In [94]:
inits [1,2,3]
tails [1,2,3]

[[],[1],[1,2],[1,2,3]]

[[1,2,3],[2,3],[3],[]]

In [95]:
search :: (Eq a) => [a] -> [a] -> Bool  
search needle haystack =  
  let nlen = length needle  
  in foldl (\acc x -> if take nlen x == needle then True else acc) False (tails haystack)

In [96]:
search [1,2] [1,2,3]

True

对搜索的 List 调用 tails，然后遍历每个 List 来检查它是不是我们想要的.

由此我们便实现了一个类似 isInfixOf 的函数，isInfixOf 从一个 List 中搜索一个子 List，若该 List 包含子 List，则返回 True

isPrefixOf 与 isSuffixOf 分别检查一个 List 是否以某子 List 开头或者结尾

In [100]:
"cat" `isInfixOf` "im a cat burglar"
"Cat" `isInfixOf` "im a cat burglar"
"cat" `isPrefixOf` "im a cat burglar"
"cat" `isPrefixOf` "cat burglar"


True

False

False

True

elem 与 notElem 检查一个 List 是否包含某元素.


- partition 

取一个限制条件和 List 作参数，返回两个 List，第一个 List 中包含所有符合条件的元素，而第二个 List 中包含余下的.

In [102]:
partition (`elem` ['A'..'Z']) "BOBsidneyMORGANeddy"
span (`elem` ['A'..'Z']) "BOBsidneyMORGANeddy" -- span 和 break 会在遇到第一个符合或不符合条件的元素处断开，而 partition 则会遍历整个 List

partition (>3) [1,3,5,6,3,2,1,0,3,7] 

("BOBMORGAN","sidneyeddy")

("BOB","sidneyMORGANeddy")

([5,6,7],[1,3,3,2,1,0,3])

- find

find 取一个 List 和限制条件作参数，并返回首个符合该条件的元素，而这个元素是个 Maybe 值

In [105]:
:t find
find (>5) [1,2,5,4,6,7]
find (>9) [1,2,5,4,6,7]

find :: forall (t :: * -> *) a. Foldable t => (a -> Bool) -> t a -> Maybe a

Just 6

Nothing

Maybe 型的值只能为空或者单一元素，而 List 可以为空,一个元素，也可以是多个元素

head (dropWhile (\(val,y,m,d) -> val < 1000) stock) 。但 head 并不安全! 如果我们的股票没涨过 1000 会怎样? dropWhile 会返回一个空 List，而对空 List 取 head 就会引发一个错误。把它改成 find ( \ (val,y,m,d) -> val > 1000) stock 就安全多啦，若存在合适的结果就得到它, 像 Just (1001.4,2008,9,4)，若不存在合适的元素(即我们的股票没有涨到过 1000)，就会得到一个 Nothing.

- elemIndex

elemIndex 与 elem 相似，只是它返回的不是布尔值，它只是'可能' (Maybe)返回我们找的元素的索引


In [107]:
:t elemIndex
3 `elemIndex` [1,2,3,4,5]
9 `elemIndex` [1,2,3,4,5]

elemIndex :: forall a. Eq a => a -> [a] -> Maybe Int

Just 2

Nothing

elemIndices 与 elemIndex 相似，只不过它返回的是 List，就不需要 Maybe

In [108]:
' ' `elemIndices` "Where are the spaces?"

[5,9,13]

- findIndex

findIndex 与 find 相似，但它返回的是可能存在的首个符合该条件元素的索引。findIndices 会返回所有符合条件的索引

In [109]:
findIndex (==4) [5,3,2,1,6,4]
findIndex (==9) [5,3,2,1,6,4]
findIndices (`elem` ['A'..'Z']) "Where Are The Caps?"  

Line 1: Use elemIndex
Found:
findIndex (== 4)
Why not:
elemIndex 4Line 1: Use elemIndex
Found:
findIndex (== 9)
Why not:
elemIndex 9

Just 5

Nothing

[0,6,10,14]

- zip 系列

zip 和 zipWith，它们只能将两个 List 组到一个二元组数或二参函数中，但若要组三个 List 该怎么办? 好说~ 有 zip3,zip4...,和 zipWith3, zipWith4...直到 7

In [110]:
zipWith3 (\x y z -> x + y + z) [1,2,3] [4,5,2,2] [2,2,3]
zip4 [2,3,3] [2,2,2] [5,5,3] [2,2,2]

[7,9,8]

[(2,2,5,2),(3,2,5,2),(3,2,3,2)]

- lines
lines 会非常有用。它取一个字符串作参数。并返回由其中的每一行组成的 List.

In [111]:
lines "first line\nsecond line\nthird line"

["first line","second line","third line"]

unlines 是 lines 的反函数，它取一组字符串的 List，并将其通过 '\n'合并到一块.

In [114]:
unlines ["first line","second line","third line"]

"first line\nsecond line\nthird line\n"

- words words 和 unwords 可以把一个字符串分为一组单词或运行相反的操作

In [116]:
words "hey these are the words in this sentence"
words "hey these are the words in this\nsentence"
unwords ["hey","there","mate"]

["hey","these","are","the","words","in","this","sentence"]

["hey","these","are","the","words","in","this","sentence"]

"hey there mate"

- nub

nub，它可以将一个 List 中的重复元素全部筛掉

In [120]:
import Data.List (nub) --要先import
:t nub
nub [1,2,3,4,3,2,1,2,3,4,3,2,1]
nub "Lots of words and stuff"

nub :: forall a. Eq a => [a] -> [a]

[1,2,3,4]

"Lots fwrdanu"

- delete 

delete 取一个元素和 List 作参数，会删掉该 List 中首次出现的这一元素

In [121]:
delete 'h' "hey there ghang!" 

"ey there ghang!"

In [122]:
delete 'h' . delete 'h' $ "hey there ghang!"

"ey tere ghang!"

In [123]:
delete 'h' . delete 'h' . delete 'h' $ "hey there ghang!"

"ey tere gang!"

- \

\ 表示 List 的差集操作，这与集合的差集很相似，它会从左边 List 中的元素扣除存在于右边 List 中的元素一次

In [125]:
[1..10] \\ [2,5,9]
"Im a big baby" \\ "big"

[1,3,4,6,7,8,10]

"Im a  baby"

- union

union 与集合的并集也是很相似，它返回两个 List 的并集，即遍历第二个 List 若存在某元素不属于第一个 List，则追加到第一个 List

- intersect

相当于集合的交集

In [127]:
"hey man" `union` "man what's up" 
[1..7] `union` [5..10]
[1..7] `intersect` [5..10]

"hey manwt'sup"

[1,2,3,4,5,6,7,8,9,10]

[5,6,7]

- insert

insert 可以将一个元素插入一个可排序的 List，并将其置于首个大于等于它的元素之前，如果使用 insert 来给一个排过序的 List 插入元素，返回的结果依然是排序的

In [128]:
insert 4 [1,2,3,5,6,7] 
insert 'g' $ ['a'..'f'] ++ ['h'..'z'] 
insert 3 [1,2,4,3,2,1]

[1,2,3,4,5,6,7]

"abcdefghijklmnopqrstuvwxyz"

[1,2,3,4,3,2,1]

许多函数都有一个共同点，那就是参数中（或者返回值）都有个Int

Data.List 中包含了更通用的替代版，如: genericLength，genericTake

In [131]:
:t length
:t (/)
let xs = [1..6] in sum xs / length xs

length :: forall (t :: * -> *) a. Foldable t => t a -> Int

(/) :: forall a. Fractional a => a -> a -> a

<interactive>:1:20: error:
    • No instance for (Fractional Int) arising from a use of ‘/’
    • In the expression: sum xs / length xs
      In the expression: let xs = [1 .. 6] in sum xs / length xs
      In an equation for ‘it’: it = let xs = ... in sum xs / length xs

In [132]:
:t genericLength
let xs = [1..6] in sum xs/ genericLength xs

genericLength :: forall i a. Num i => [a] -> i

3.5

nub, delete, union, intsect 和 group 函数也有各自的通用替代版 nubBy，deleteBy，unionBy，intersectBy 和 groupBy，它们的区别就是前一组函数使用 (==) 来测试是否相等，而带 By 的那组则取一个函数作参数来判定相等性，

group 就与 groupBy (==) 等价

有个记录某函数在每秒的值的 List，而我们要按照它小于零或者大于零的交界处将其分为一组子 List。如果用 group，它只能将相邻并相等的元素组到一起，而在这里我们的标准是它们是否互为相反符号。groupBy 登场! 它取一个含两个参数的函数作为参数来判定相等性

In [136]:
let values = [-4.3, -2.4, -1.2, 0.4, 2.3, 5.9, 10.5, 29.1, 5.3, -2.4, -14.5, 2.9, 2.3]
groupBy(\ x y -> (x > 0) == (y > 0) ) values

[[-4.3,-2.4,-1.2],[0.4,2.3,5.9,10.5,29.1,5.3],[-2.4,-14.5],[2.9,2.3]]

- on

Data.Function 中还有个 on 函数可以让它的表达更清晰，其定义如下:

In [144]:
import Data.Function as f

Parse error (line 1, column 25): parse error on input ‘f’

In [142]:
import Data.Function as F --as 的module别名也要大写，不然会报错
:t F.on

F.on :: forall b c a. (b -> b -> c) -> (a -> b) -> a -> a -> c

on :: (b -> b -> c) -> (a -> b) -> a -> a -> c  
f \`on\` g = \x y -> f (g x) (g y)

In [145]:
-- groupBy(\ x y -> (x > 0) == (y > 0) ) values
groupBy ((==) `on` (> 0)) values 

[[-4.3,-2.4,-1.2],[0.4,2.3,5.9,10.5,29.1,5.3],[-2.4,-14.5],[2.9,2.3]]

- sortBy

List 是可以比较大小的，且比较的依据就是其中元素的大小。如果按照其子 List 的长度为标准当如何?

In [147]:
let xs = [[5,4,5,4,4],[1,2,3],[3,5,4,3],[],[2],[2,2]]
sortBy ( compare `on` length ) xs
-- \x y -> length x `compare` length y

[[],[2],[2,2],[1,2,3],[3,5,4,3],[5,4,5,4,4]]

与带 By 的函数打交道时，若要判断相等性，则 (==) \`on\` something。

若要判定大小，则 compare \`on\` something

# Data.Char

Data.Char模块中含有一系列用于判定字符范围的函数

# Data.Map

在 Haskell 中表示关联列表的最简单方法就是弄一个二元组的 List，而这二元组就首项为键，后项为值

In [172]:
import qualified Data.Map as Map hiding (foldr)
phoneBook = [("betty","555-2938") ,
             ("bonnie","452-2928") ,
             ("patsy","493-2928") ,
             ("lucille","205-2928") ,
             ("wendy","939-8282") ,
             ("penny","853-2492") ]

In [162]:
findKeyFirst :: (Eq k) => k -> [(k,v)] -> v 
findKeyFirst key xs = snd . head . Prelude.filter (\(k,v) -> key == k) $ xs

In [163]:
findKeyFirst "aa" phoneBook

Prelude.head: empty list

为了避免程序崩溃，如果没找到相应的键，就返回 Nothing。而找到了就返回 Just something。而这 something 就是键对应的值

In [164]:
findKeySnd :: (Eq k) => k -> [(k,v)] -> Maybe v 
findKeySnd key [] = Nothing
findKeySnd key ((k,v):xs) = 
     if key == k then 
         Just v 
     else 
         findKeySnd key xs

In [165]:
findKeySnd "aa" phoneBook

Nothing

用fold实现

In [170]:
findKey :: (Eq k) => k -> [(k,v)] -> Maybe v 
findKey key = foldr (\(k,v) acc -> if key == k then Just v else acc) Nothing

In [171]:
findKey "betty" phoneBook

Just "555-2938"

- fromList (构建Map的方法）


fromList 取一个关联列表，返回一个与之等价的 Map

In [173]:
Map.fromList [("betty","555-2938"),("bonnie","452-2928"),("lucille","205-2928")] 

fromList [("betty","555-2938"),("bonnie","452-2928"),("lucille","205-2928")]

In [174]:
Map.fromList [(1,2),(3,4),(3,2),(5,5)] 

fromList [(1,2),(3,2),(5,5)]

若其中存在重复的键,就将其忽略

Key 应该是可排序的，因为Data.Map 模块中是强制的。因为它会按照某顺序将其组织在一棵树中.在处理键值对时，只要键的类型属于 Ord 类型类，就应该尽量使用Data.Map.empty 返回一个空 map

- insert

insert 取一个键，一个值和一个 map 做参数，给这个 map 插入新的键值对，并返回一个新的 map

In [175]:
Map.insert 5 600 (Map.insert 4 200 ( Map.insert 3 100  Map.empty))
Map.insert 5 600 . Map.insert 4 200 . Map.insert 3 100 $ Map.empty 

fromList [(3,100),(4,200),(5,600)]

fromList [(3,100),(4,200),(5,600)]

In [176]:
fromList' :: (Ord k) => [(k,v)] -> Map.Map k v 
fromList' = foldr (\(k,v) acc -> Map.insert k v acc) Map.empty

 从一个空的 map 开始，然后从右折叠，随着遍历不断地往 map 中插入新的键值对.



null 检查一个 map 是否为空.

In [177]:
null Map.empty

True

size 检查map的大小

In [178]:
Map.size $ Map.fromList [(2,4),(3,3),(4,2),(5,4),(6,4)]

5

- singleton (构建Map）

singleton 取一个键值对做参数,并返回一个只含有一个映射的 map

In [179]:
Map.singleton 5 5

fromList [(5,5)]

lookup 与 Data.List 的 lookup 很像,只是它的作用对象是 map，如果它找到键对应的值。就返回 Just something，否则返回 Nothing


- member

member 是个判断函数，它取一个键与 map 做参数，并返回该键是否存在于该 map

In [181]:
Map.member 3 $ Map.fromList [(3,6),(4,3),(6,9)]

True

map 与 filter 与其对应的 List 版本很相似，判断与对value进行操作

- toList

toList 是 fromList 的反函数

In [183]:
Map.toList . Map.insert 9 2 $ Map.singleton 4 3 

[(4,3),(9,2)]

- keys 和 elems

keys 与 elems 各自返回一组由键或值组成的 List，keys 与 map fst . Map.toList 等价，elems 与 map snd . Map.toList等价

- fromListWith 类似fromList， 但不忽略重复的key

In [184]:
phoneBook =   
    [("betty","555-2938")  
    ,("betty","342-2492")  
    ,("bonnie","452-2928")  
    ,("patsy","493-2928")  
    ,("patsy","943-2929")  
    ,("patsy","827-9162")  
    ,("lucille","205-2928")  
    ,("wendy","939-8282")  
    ,("penny","853-2492")  
    ,("penny","555-2111")  
    ]

如果用 fromList 来生成 map，我们会丢掉许多号码

In [185]:
phoneBookToMap :: (Ord k) => [(k, String)] -> Map.Map k String  
phoneBookToMap xs = Map.fromListWith (\number1 number2 -> number1 ++ ", " ++ number2) xs

In [186]:
Map.lookup "patsy" $ phoneBookToMap phoneBook

Just "827-9162, 943-2929, 493-2928"

一旦出现重复键，这个函数会将不同的值组在一起，同样，也可以缺省地将每个值放到一个单元素的 List 中，再用 ++ 将他们都连接在一起

In [189]:
phoneBookToMap :: (Ord k) => [(k, a)] -> Map.Map k [a] --k [a]
phoneBookToMap xs = Map.fromListWith (++) $ map (\(k,v) -> (k,[v])) xs 
Map.lookup "patsy" $ phoneBookToMap phoneBook 


Just ["827-9162","943-2929","493-2928"]

遇到重复元素时，单选最大的那个值

In [190]:
Map.fromListWith max [(2,3),(2,5),(2,100),(3,29),(3,22),(3,11),(4,22),(4,15)]

fromList [(2,100),(3,29),(4,22)]

或是将相同键的值都加在一起

In [191]:
Map.fromListWith (+) [(2,3),(2,5),(2,100),(3,29),(3,22),(3,11),(4,22),(4,15)]

fromList [(2,108),(3,62),(4,37)]

In [192]:
Map.insertWith (+) 3 100 $ Map.fromList [(3,4),(5,103),(6,339)]

fromList [(3,104),(5,103),(6,339)]

Data.Map 里面还有不少函数，[http://www.haskell.org/ghc/docs/latest/html/libraries/containers/Data-Map.html 这个文档]中的列表就很全了

# Data.Set

对一个集合而言，最常见的操作莫过于并集，判断从属或是将集合转为 List.
由于 Data.Set 模块与 Prelude 模块和 Data.List 模块中存在大量的命名冲突，所以我们使用 qualified import

In [195]:
import qualified Data.Set as Set

大同小异了

In [196]:
Set.fromList [2,3,4] `Set.isSubsetOf` Set.fromList [1,2,3,4,5] 

True

集合有一常见用途，那就是先 fromList 删掉重复元素后再 toList 转回去。尽管 Data.List 模块的 nub 函数完全可以完成这一工作，但在对付大 List 时则会明显的力不从心。使用集合则会快很多，nub 函数只需 List 中的元素属于 Eq 类型类就行了，而若要使用集合，它必须得属于 Ord 类型类

In [197]:
let setNub xs = Set.toList $ Set.fromList xs 

In [198]:
setNub "HEY WHATS CRACKALACKIN"

" ACEHIKLNRSTWY"

In [201]:
nub "HEY WHATS CRACKALACKIN"

"HEY WATSCRKLIN"

# 创建自己的模块

先从新建一个 Geometry.hs 的文件开始.
在模块的开头定义模块的名称，如果文件名叫做 Geometry.hs 那它的名字就得是 Geometry

In [203]:
module Geometry  
( sphereVolume  
,sphereArea  
,cubeVolume  
,cubeArea  
,cuboidArea  
,cuboidVolume  
) where  

sphereVolume :: Float -> Float  
sphereVolume radius = (4.0 / 3.0) * pi * (radius ^ 3)  

sphereArea :: Float -> Float  
sphereArea radius = 4 * pi * (radius ^ 2)  

cubeVolume :: Float -> Float  
cubeVolume side = cuboidVolume side side side  

cubeArea :: Float -> Float  
cubeArea side = cuboidArea side side side  

cuboidVolume :: Float -> Float -> Float -> Float  
cuboidVolume a b c = rectangleArea a b * c  

cuboidArea :: Float -> Float -> Float -> Float  
cuboidArea a b c = rectangleArea a b * 2 + rectangleArea a c * 2 + rectangleArea c b * 2  

rectangleArea :: Float -> Float -> Float  
rectangleArea a b = a * b

In [204]:
import Geometry

将 Geometry.hs 文件至于用到它的进程文件的同一目录之下.
模块也可以按照分层的结构来组织，每个模块都可以含有多个子模块。而子模块还可以有自己的子模块。我们可以把 Geometry 分成三个子模块，而一个模块对应各自的图形对象.
首先，创建一个 Geometry 文件夹，注意首字母要大写，在里面新建三个文件
如下就是各个文件的内容:

In [206]:
module Geometry.Sphere  
( volume  
, area  
) where  

volume :: Float -> Float  
volume radius = (4.0 / 3.0) * pi * (radius ^ 3)  

area :: Float -> Float  
area radius = 4 * pi * (radius ^ 2)

In [208]:
module Geometry.Cuboid  
( volume  
, area  
) where  

volume :: Float -> Float -> Float -> Float  
volume a b c = rectangleArea a b * c  

area :: Float -> Float -> Float -> Float  
area a b c = rectangleArea a b * 2 + rectangleArea a c * 2 + rectangleArea c b * 2  

rectangleArea :: Float -> Float -> Float  
rectangleArea a b = a * b

In [209]:
module Geometry.Cube  
( volume  
,area  
) where  

import qualified Geometry.Cuboid as Cuboid  

volume :: Float -> Float  
volume side = Cuboid.volume side side side  

area :: Float -> Float  
area side = Cuboid.area side side side

qualified import 来避免重名

In [210]:
import qualified Geometry.Sphere as Sphere  
import qualified Geometry.Cuboid as Cuboid  
import qualified Geometry.Cube as Cube

In [211]:
Sphere.area 3

113.097336

In [212]:
Sphere.volume 3

113.097336

In [214]:
Cuboid.area 3 3 3

54.0